(myst-nb-usage)=
# MyST-NB Usage

The (probably) most useful function of this library is the `myst_nb_metadata_injector`.
This allows you to write _magic-comments_ inside of your Jupyter code cells to inject metadata into those cells and configure the execution/rendering behavior of the  {{mystnb}} library.

For example, with a simple comment like `# remove-input` you can remove the code cell in your rendered documentation and only show the output without having to fight with setting the metadata tags of your code cell. See the following example:

Source (what you would write inside of your Jupyter notebook)
```python
# remove-input
print("My code cell is removed")
```

Rendered output of MyST-NB:

In [ ]:
# remove-input
print("My code cell is removed")

As you can (can't ?) see the output of the cell was rendered but the code-cell itself is hidden on the generated page.

The main motivation for this functionality is to keep the configuration _local_ and not hide it behind an `metadata` window inside of the Jupyter Notebook UI.
This is especially frustrating for users that use UIs that do not support configuring the code cell metadata, like VSCode.


## Enabling the Metadata Injector for MyST-NB

To enable {{mystnb}} to use the metadata injection preprocessor by default, you have to add the following code to your [conf.py](https://www.sphinx-doc.org/en/master/usage/configuration.html):


```{literalinclude} conf.py
---
language: python
start-after: enable-start
end-before: enable-end
---
```

The current version requires _you_ to specify the `prefix` that indicates the start of a potential _magic comment_.
Usually, this should be comment symbol(s) of the programming language.
By default, and used in the example above, the `prefix=#` as this is the comment symbol for the Python programming language.
The `delimiter` may be a character/string of your choice and will be used for separating key-value pairs.

To see a list of all possible configuration, see the [API documentation of mystnb_nb_metadata_injector](myst_nb_metadata_injector).

:::{important}
To overwrite the default `ipynb` extension parser, you are required to use `MyST-NB>=0.14.0`!
At the time of writing, the [jupyter-book project](https://jupyterbook.org/en/stable/intro.html) (`v0.13`) fixes the `MyST-NB` version to `0.13.1` and _cannot_ overwrite the default `ipynb` format.

You can rename your notebooks with a different extension (only characters after the last `.` are considered the extension) and use that option for the custom parser instead.
The main issue with that approach is that some tools have a hard time working with a different extension for Jupyter notebooks (like VSCode).
:::

## Understanding the examples

To show the _effects_ of the different options, the article will be structured as follows:

1. The jupyter-notebook cell's source code is shown. The visual indicator of the _source_ code of the code cell, is that the stylized code-cell has _no_ border around it.
2. The jupyter-notebook cell's source code is displayed _after_ applying the preprocessor. This will remove the magic comment and inject the metadata into the cell's code (not shown here). The processed code cell is stylized with a white border and a green left-border. This is the rendered code cell you would be seeing in your documentation.
3. The output of the executed code-cell is shown with a white bordered cell.

And one last visual example to clarify the different _stages_:

Source:
```python
# I am stage 1 and show the 'source' code of
# the code cell
```

Rendered:

In [ ]:
# magic-comment
"""
I am stage 2 and show the post-processed code cell.
If the magic comment matches one of the MyST-NB tags
it would be removed from this cell and injected into the
metadata field.
"""
print("I am the rendered code cell output")

The {{mystnb}} library provides two ways to configure the execution/rendering process of a code cell.
One is to _add_ specific strings to the `tags` metadata list.
See [](tags-conf) for details on what is possible with this configuration method.
The other configuration method is to _set_ key-value pairs inside of the `mystnb` metadata tag of the code-cell.
See [](mystnb-conf) for details.
Note that the things you can configure are _mostly_ different between both of these methods! You should take a look at _both_!

If the last paragraph confused you, don't worry.
It is probably not that important for you as it discusses the _conceptual_ difference between the two configuration options.
The important thing to know is that the `tag` based configuration only allows you to _add_ a string, while the other allows you to set a key-value pair.
You will see how this effects the configuration in the following sections.

(tags-conf)=
## Tag configuration

At the time of writing, the following configuration keywords can be set in the `tags` metadata entry of a Jupyter code-cell:

- [](remove-input)
- [](remove-stderr)
- [](remove-output)
- [](remove-cell)
- [](skip-execution)
- [](raises-exception)

For more details, see the `Cell tags` section of the {external+mystnb:doc}`MyST configuration documentation <configuration>`.


### no-match example

Example showing what happens with code cells that do _not_ include magic comments/are _normal_ code cells.

Source:
```python
# unknown
print("Example showing what happens if the cell doesn't include a magic comment")
```

Rendered:

In [ ]:
# unknown
print("Example showing what happens if the cell doesn't include a magic comment")

Not so surprisingly, nothing much happens with those code-cells. They are simply executed and rendered as-is.

(remove-input)=
### remove-input

Removes the source code of the cell.
Useful if you are only interested in the output.

Source:
```python
# remove-input
print("My code cell is removed")
```

Rendered:

In [ ]:
# remove-input
print("My code cell is removed")

(remove-stderr)=
### remove-stderr

Only remove the standard error output of the code cell.
This is useful if an underlying library uses `stderr` to print progress or other debug messages that would only pollute the output for the documentation.

Source:
```python
# remove-stderr
import sys
print("The stderr output is removed", file=sys.stderr)
print("But not the default output")
```

Rendered:

In [ ]:
# remove-stderr
import sys

print("The stderr output is removed", file=sys.stderr)
print("But not the default output")

(remove-output)=
### remove-output

Remove the output of the code cell (duh).
Useful to hide some warning during important without having filter the messages or to hide visual output.

Source:
```python
# remove-output
import sys
print("My output is removed")
print("including the error output", file=sys.stderr)
```

Rendered:

In [ ]:
# remove-output
import sys

print("My output is removed")
print("including the error output", file=sys.stderr)

(remove-cell)=
### remove-cell

Remove the input as well as the output of the code cell.

Source:
```python
# remove-cell
print("This entire cell is removed")
```

Rendered:

In [ ]:
# remove-cell
print("This entire cell is removed")

As you can (can't ?) see the cell is missing from the documentation page.

```{note}
The code-cell is still executed under the hood!
```

(raises-exception)=
### raises-exception

The following code raises an exception and has 
`raises-exception` set to signal that the exception  is expected and continues with the execution and build process of the documentation.
This is helpful, if you are showing examples of _failing_ code and want to include the trace-back.

Source:
```python
# raises-exception
1 / 0
```

Rendered:

In [ ]:
# raises-exception
1 / 0

(skip-execution)=
### skip-execution

The following code _would_ raise an exception but the execution of the cell is skipped and not exception is raised.


Source:
```python
# skip-execution
1 / 0
```

Rendered:

In [ ]:
# skip-execution
1 / 0

Notice how no output is produced and no exception is raised.

(mystnb-conf)=
## Myst-NB Metadata Configuration

At the time of writing, the following configuration keyword-value pairs can be set in the `mystnb` metadata entry of a Jupyter code-cell:

- [](sec/remove_code_source)
- [](sec/remove_code_outputs)
- [](sec/number_source_lines)
- [](sec/output_stderr)
- [](sec/merge_streams)
- [](sec/text_lexer)
- [](sec/error_lexer)
- [](sec/markdown_format)
- [](sec/image)
- [](sec/figure)


In contrast to the `tag`-based configuration, the `mystnb`-based configuration uses a key-value pair!
The `delimiter` is used to separate the key-value pairs.
Note that spaces around the `delimiter` are allowed/ignored during the parsing process.
The value is **parsed as a [yaml](https://en.wikipedia.org/wiki/YAML)-string**!
This allows you to even set nested structures in a single line and is required for coercing the data to the correct type.

:::{important}
The value of the key-value pair is parsed as a _yaml_-string to allow correctly coercing the data to the desired type and to allow nested structures within a line.

This library _only_ allows setting the value on the same line. This means that you _cannot_ write the value across multiple lines!
:::


(sec/remove_code_source)=
### remove_code_source

Removes the code cell... (Surprising, right?)

Source:
```python
# remove_code_source = true
print("My code cell is removed")
```

Rendered:

In [ ]:
# remove_code_source = true
print("My code cell is removed")

(sec/remove_code_outputs)=
### remove_code_outputs

Removes the output of the cell, including the output to standard error.

Source:
```python
# remove_code_outputs = true
import sys
print("My output is removed")
print("including the error output", file=sys.stderr)
```

In [ ]:
# remove_code_outputs = true
import sys

print("My output is removed")
print("including the error output", file=sys.stderr)

(sec/number_source_lines)=
### number_source_lines
Add source line numbering to the current cell.

Source:
```python
# number_source_lines = true
# line 1
# line 2
print("I love my husky.")
```

Rendered:

In [ ]:
# number_source_lines = true
# line 1
# line 2
print("I love my husky.")

(sec/output_stderr)=
### output_stderr

Define the behavior for standard error output.
Can be set to one of:
- `show`: (Default) Show the stderr as a separate output
- `remove`: Hide stderr
- `remove-warn`: Hide stderr but log a warning to Sphinx
- `warn|error|severe`: Log the error with a specific logging level to Sphinx

Source:
```python
# output_stderr = show
import sys
print("stdout 0")
print("stderr", file=sys.stderr)
print("stdout 1")
```

Rendered:

In [ ]:
# output_stderr = show
import sys

print("stdout 0")
print("stderr", file=sys.stderr)
print("stdout 1")

Source:
```python
# output_stderr = remove
import sys
print("stdout 0")
print("stderr", file=sys.stderr)
print("stdout 1")
```

In [ ]:
# output_stderr = remove
import sys

print("stdout 0")
print("stderr", file=sys.stderr)
print("stdout 1")

For the other options, see the official [MyST-NB documentation for the stderr configuration](https://myst-nb.readthedocs.io/en/latest/render/format_code_cells.html#remove-stdout-or-stderr).

(sec/merge_streams)=
### merge_streams

Merge standard output and standard error into a single output cell.

:::{error}
This doesn't work!

I will have to investigate why this isn't working.
It seems like it also doesn't work by manually setting the tag.
:::

Source:
```python
# merge_streams = true
import sys
print("stdout 0")
print("stderr", file=sys.stderr)
print("stdout 1")
```

Rendered:

In [ ]:
# merge_streams = true
import sys

print("stdout 0")
print("stderr", file=sys.stderr)
print("stdout 1")

(sec/text_lexer)=
### text_lexer

Pygments lexer applied to standard out, standard error and text/plain outputs.

Source:
```python
# text_lexer = python
print("# Render output with Python code cell lexer")
```

Rendered:

In [ ]:
# text_lexer = python
print("# Render output with Python code cell lexer")

(sec/error_lexer)=
### error_lexer
Pygments lexer applied to the error/traceback outputs.
Note, that to be able to correctly build the documentation, the following example also uses [](raises-exception) to let the executor now that we _expect_ an exception to be raised.

Source:
```python
# error_lexer = python
# raises-exception
raise Exception("# The last line be lexed as a Python comment, everything else will be broken")
```

Rendered:

In [ ]:
# error_lexer = python
# raises-exception
raise Exception(
    "# The last line be lexed as a Python comment, everything else will be broken"
)

(sec/markdown_format)=
### markdown_format
The format to use for text/markdown rendering.

Can be one of:
- `commonmark` (Default)
- `gfm`
- `myst`

Source:
```python
# markdown_format = myst
from IPython.display import Markdown

Markdown(r"""
Term 1
: Requires `deflist` extension to be loaded
""")
```


Rendered:

In [ ]:
# markdown_format = myst
from IPython.display import Markdown

Markdown(
    r"""
Term 1
: Requires myst and `deflist` extension to be loaded
"""
)

In contrast to the default with

Source:
```python
# markdown_format = commonmark
from IPython.display import Markdown

Markdown(r"""
Term 1
: Won't render as `deflist` with commonmark
""")
```

In [ ]:
# markdown_format = commonmark
from IPython.display import Markdown

Markdown(
    r"""
Term 1
: Won't render as `deflist` with commonmark
"""
)

(sec/image)=
### image

Options for image outputs.

To see a list of all possible configuration values, take a look at the [official image documentation](https://myst-nb.readthedocs.io/en/latest/render/format_code_cells.html#images-and-figures).

Source:

```python
# image = {"width": 20%, "align": "right", "alt": "A cute comic-style fish with a party hat"}
from IPython.display import Image
Image("fish.png")
```

In [ ]:
# image = {"width": 20%, "align": "right", "alt": "A cute comic-style fish with a party hat"}
from IPython.display import Image

Image("fish.png")

(sec/figure)=
### figure

Options for figure outputs.
To see a list of all possible configuration values, take a look at the [official figure documentation](https://myst-nb.readthedocs.io/en/latest/render/format_code_cells.html#images-and-figures).

Source:

```python
# figure = {"caption": "I am a top caption!", "caption_before": true}
from IPython.display import Image
Image("fish.png")
```

In [ ]:
# figure = {"caption": "I am a top caption!", "caption_before": true}
from IPython.display import Image

Image("fish.png")